In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
import pandas as pd
from datasets import Dataset

In [58]:
data = pd.read_csv(r'./IMDB Dataset.csv/IMDB Dataset.csv')

In [59]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [44]:
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size =0.3)
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 15000
    })
})

In [45]:
#input_ids, attention_mask, label--->
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [46]:
label2id ={'negative':0,'positive':1}
id2label = {0:'negative', 1:'positive'}
dataset = dataset.map(lambda x: {'label':label2id[x['sentiment']]})

Map: 100%|██████████| 15000/15000 [00:00<00:00, 22081.75 examples/s]


In [47]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'label'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment', 'label'],
        num_rows: 15000
    })
})

In [48]:
dataset['train'][0]

{'review': "This is by far THE WORST movie i have ever watched. I've seen some pretty awful movies in my time but this ones takes the cake, no, wait, i mean the the whole damn bakery. It is so bad that i believe a word to describe the way you will feel after watching this atrocity has yet to be created. Please just do yourself a favor, if you ever get the urge to watch this and watch thirty minutes of that annoying purple dinosaur Barney, then multiply that thirty times fold and you would still only get a small fraction of the horror you would be in store for. In summation, i guess you really can call it a horror movie, but only if you're willing to be scared senseless by the worst acting in the business and utterly pointless story.<br /><br />Real Rating, -10 Disgusting",
 'sentiment': 'negative',
 'label': 0}

In [49]:
from transformers import AutoTokenizer

import torch

device= torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_ckpt = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

In [50]:
tokenizer

BertTokenizerFast(name_or_path='huawei-noah/TinyBERT_General_4L_312D', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [51]:
def tokenize(batch):
    temp = tokenizer(batch['review'], padding =True, truncation = True,max_length=300)
    return temp

dataset= dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map: 100%|██████████| 15000/15000 [00:12<00:00, 1230.22 examples/s]


In [52]:
dataset['train'][0].keys()

dict_keys(['review', 'sentiment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [53]:
# ?!pip install evaluate

import evaluate
import numpy as np

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)




In [54]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(label2id), label2id=label2id, id2label=id2label)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [56]:
args = TrainingArguments(
    output_dir='train_dir',
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy='epoch'

)

trainer = Trainer(
    model=model,
    args= args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [57]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3197152614593506,
 'eval_accuracy': 0.8828666666666667,
 'eval_runtime': 140.0279,
 'eval_samples_per_second': 107.122,
 'eval_steps_per_second': 3.349,
 'epoch': 3.0}

In [ ]:
trainer.save_model('tinybert-sentiment-analysis')

In [ ]:
data = ['this movie was horrible, the plot was really boring.acting was okay',
        'the movie is really sucked.there is not plot and acting was bad',
        'what a beautiful movie.great plot. acting was good. will see it again']

In [ ]:
from transformers import pipeline

classifier = pipeline('text-classification', model='tinybert-sentiment-analysis', device= device)

classifier(data)

Device set to use cpu


[{'label': 'negative', 'score': 0.9941874742507935},
 {'label': 'negative', 'score': 0.9938879609107971},
 {'label': 'positive', 'score': 0.9946723580360413}]

In [ ]:
# import boto3

# s3 =boto3.client('s3')

# bucket_name='mlops-vjr'

# def create_bucket(bucket_name):
#     response=s3.list_buckets()
#     buckets = [buck['NAme'] for buck in response['Buckets']]
#     if bucket_name not in buckets:
#         s3.create_bucket(Bucket=bucket_name)
#         print("Bucket is created")

#     else:
#         print('Bucket already exists in you account')

# create_bucket(bucket_name)

### PUSH TO S# BUCKET MODEL WEIGHTS

In [39]:
import os
import boto3
s3 = boto3.client('s3')
bucket_name= 'mlops-vjr'

def upload_directory(directory_path, s3_prefix):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file).replace("\\","/")
            relpath = os.path.relpath(file_path, directory_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\","/")

            s3.upload_file(file_path, bucket_name, s3_key)

upload_directory('tinybert-sentiment-analysis','ml-models/tinybert-sentiment-analysis')
